In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# See http://drivendata.github.io/cookiecutter-data-science/
import os
import sys

src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [3]:
import pickle

import numpy as np
import pandas as pd

# Set the seed for Keras
np.random.seed(4)

from data.data import get_data, noise_bagging
from features.features import Features

from model.model import split_train_test

Using Theano backend.


In [4]:
# Define the properties of this run

# Set the number of time steps that will be used in for the RNN
ts_steps = 4

# Set the radius in km that will be used to compute the nest count of neighbouring nests
nestCount_radius = 200

# Set the radius in which the krill data will be reported for each nest
krill_radius = 100

# Set the area which is used for computing the sea ice feature
padding = 1

In [5]:
# Get the data with the specified time lag. The time series is assembled as relative difference
# betweene the successive steps.
df_features, scaler = get_data(ts_steps)
df_features.head()

y_true inferred_y_true inferred_t  t0   t1  \
site_id species           year                                               
ACUN    adelie penguin    1983  1.0000            True       True NaN  1.0   
        chinstrap penguin 1983  1.0000            True      False NaN  1.0   
ADAM    adelie penguin    1983  1.0000            True       True NaN  1.0   
ADAR    adelie penguin    1983  1.0331            True      False NaN  1.0   
AILS    chinstrap penguin 1983  1.0000            True      False NaN  1.0   

                                 t2   t3  countError  
site_id species           year                        
ACUN    adelie penguin    1983  1.0  1.0    0.707107  
        chinstrap penguin 1983  1.0  1.0    0.707107  
ADAM    adelie penguin    1983  1.0  1.0    0.707107  
ADAR    adelie penguin    1983  1.0  1.0    0.196116  
AILS    chinstrap penguin 1983  1.0  1.0    0.707107

In [6]:
# Remove the first occurence
df_features.dropna(inplace=True)

In [7]:
# Add the features to the DataFrame
features = Features(krill_radius, nestCount_radius, padding)
df_features = features.add_features(df_features)

df_features.head()


Loading sea ice distMat from data/interim/
Computing sea ice distMat and caching result in data/interim/
This can take a while.. (apologies for computing this via brute force)
Done.
Computing krill distMat and caching result in data/interim/
This can take a while.. (apologies for computing this via brute force)


y_true inferred_y_true inferred_t   t0   t1   t2  \
site_id species        year                                                     
ACUN    adelie penguin 1984     1.0            True       True  1.0  1.0  1.0   
                       1985     1.0            True       True  1.0  1.0  1.0   
                       1986     1.0            True       True  1.0  1.0  1.0   
                       1987     1.0            True       True  1.0  1.0  1.0   
                       1988     1.0            True       True  1.0  1.0  1.0   

                              t3  countError  adelie penguin  \
site_id species        year                                    
ACUN    adelie penguin 1984  1.0    0.707107             1.0   
                       1985  1.0    0.707107             1.0   
                       1986  1.0    0.707107             1.0   
                       1987  1.0    0.707107             1.0   
                       1988  1.0    0.707107             1.0   

                             chinstrap penguin    ...     sea_ice_month_3  \
site_id species        year                       ...                       
ACUN    adelie penguin 1984                0.0    ...            0.000000   
                       1985                0.0    ...            0.006667   
                       1986                0.0    ...            0.141111   
                       1987                0.0    ...            0.083611   
                       1988                0.0    ...            0.158333   

                             sea_ice_month_4  sea_ice_month_5  \
site_id species        year                                     
ACUN    adelie penguin 1984         0.000000         0.007222   
                       1985         0.000000         0.000000   
                       1986         0.000000         0.000278   
                       1987         0.000000         0.150833   
                       1988         0.129444         0.122778   

                             sea_ice_month_6  sea_ice_month_7  \
site_id species        year                                     
ACUN    adelie penguin 1984         0.177222         0.000000   
                       1985         0.060278         0.059444   
                       1986         0.043889         0.150556   
                       1987         0.168333         0.164722   
                       1988         0.000000         0.054444   

                             sea_ice_month_8  sea_ice_month_9  \
site_id species        year                                     
ACUN    adelie penguin 1984         0.000000         0.064167   
                       1985         0.016111         0.000000   
                       1986         0.065000         0.003333   
                       1987         0.000000         0.151389   
                       1988         0.158889         0.151389   

                             sea_ice_month_10  sea_ice_month_11     krill  
site_id species        year                                                
ACUN    adelie penguin 1984          0.141944          0.147778  1.051298  
                       1985          0.140000          0.165556  5.788702  
                       1986          0.171944          0.000000  0.000000  
                       1987          0.181667          0.181667  0.000000  
                       1988          0.008889          0.000000  4.264471  

[5 rows x 26 columns]

In [8]:
# Split into train and test
df_train, df_test = split_train_test(df_features)
df_train.shape, df_test.shape

((17496, 26), (1296, 26))

In [9]:
# There are a lot of missing values in the original data and I believe that the truly observed values
# should be trusted more. In order to increase the training data, noise is added to which artificially
# increase the number of observations. Here, the truly observed data is enriched relatively more to
# put more emphasise on them.

df_YObserved = df_train[~df_train['inferred_y_true']]
df_trainNoiseYObserved = noise_bagging(df_YObserved, 5*df_YObserved.shape[0], strength=0.5)
print('Noise y observed', df_trainNoiseYObserved.shape)

df_TObserved = df_train[~df_train['inferred_t']]
df_trainNoiseTObserved = noise_bagging(df_TObserved, 2*df_YObserved.shape[0], strength=0.5)
print('Noise t observed', df_trainNoiseTObserved.shape)

df_trainNoise = noise_bagging(df_train, 1*df_train.shape[0], strength=0.4)
print('Noise all', df_trainNoise.shape)

df_trainNoise = pd.concat([df_trainNoise, df_trainNoiseTObserved, df_trainNoiseYObserved])
print('Final', df_trainNoise.shape)

Noise y observed (8184, 26)
Noise t observed (6261, 26)
Noise all (34992, 26)
Final (49437, 26)


In [10]:
fname = '../data/interim/features_weighted.p'
pickle.dump([df_trainNoise, df_test, df_features, features, scaler], open(fname, 'wb'))